In [1]:
import pandas as pd


Apertura de un archivo JSON para verificar estructura

In [3]:
data_reviews=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\REVIEW-ESTADOS\reviews-estados\review-Alabama\1.json", lines=True)

In [4]:
data_reviews.shape

(150000, 8)

In [3]:
data_reviews.head(2)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.140438e+20,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,1.160090e+20,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf


In [5]:
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     82943 non-null   object 
 5   pics     3087 non-null    object 
 6   resp     20893 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB


In [7]:
#ETL
#eliminado de columnas PICS y RESP
data_reviews.drop(columns=["pics", "resp"], inplace=True)
# Convertir la columna 'time' a formato de fecha y hora
data_reviews['time'] = pd.to_datetime(data_reviews['time'], unit='ms')

for columna in data_reviews.columns:
    if data_reviews[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
        data_reviews[columna] = data_reviews[columna].fillna("No data").str.strip()
    elif data_reviews[columna].dtype in ['int64', 'float64']:
        # Si la columna es numérica, llenar los valores nulos con 0
        data_reviews[columna] = data_reviews[columna].fillna(0)

# Eliminar duplicados basados en las columnas 'gmap_id' y 'user_id'
data_reviews.drop_duplicates(subset=['gmap_id', 'user_id'], inplace=True)
#cambiar tipo dato columna user_id a string
data_reviews['user_id'] = data_reviews['user_id'].astype(str)

        

In [9]:
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147686 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   user_id  147686 non-null  object        
 1   name     147686 non-null  object        
 2   time     147686 non-null  datetime64[ns]
 3   rating   147686 non-null  int64         
 4   text     147686 non-null  object        
 5   gmap_id  147686 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 7.9+ MB


In [10]:
df_aux=data_reviews.sample(n=50000, random_state=42)

In [ ]:
df_aux.to_json("review.json", index=False)

In [ ]:


def ETL_reviews_bigquery(cloud_event):
    try:
        data = cloud_event.data
        event_id = cloud_event["id"]
        event_type = cloud_event["type"]

        bucket = data["bucket"]
        name = data["name"]
        timeCreated = data["timeCreated"]
        updated = data["updated"]

        # Obtén la carpeta en el bucket desde los parámetros de la URL
        lista_carpetas=name.split("/")
        estado = lista_carpetas[2]
        estado=estado.split("-")
        estado=estado[1]

        # Crea un cliente de Cloud Storage
        storage_client = storage.Client()

        file_path = f'gs://{bucket}/{name}'
        # Convierte la lista de objetos JSON en un DataFrame de Pandas
        df = pd.read_json(file_path, lines=True)

        print("leyo bien")

        #ETL
        #eliminado de columnas PICS y RESP
        df.drop(columns=["pics", "resp"], inplace=True)
        # Convertir la columna 'time' a formato de fecha y hora
        df['time'] = pd.to_datetime(df['time'], unit='ms')

        for columna in df.columns:
            if df[columna].dtype == 'object':
                # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
                df[columna] = df[columna].fillna("No data").str.strip()
            elif df[columna].dtype in ['int64', 'float64']:
                # Si la columna es numérica, llenar los valores nulos con 0
                df[columna] = df[columna].fillna(0)
        
        # Eliminar duplicados basados en las columnas 'gmap_id' y 'user_id'
        df.drop_duplicates(subset=['gmap_id', 'user_id'], inplace=True)
        #cambiar tipo dato columna user_id a string
        df['user_id'] = df['user_id'].astype(str)

        #agregamos la columna state para indicar de que estado proviene cada review
        df['state'] = estado

        #iniciamos el cliente de BigQuery
        client = bigquery.Client()
        
        project_id='dms-pfh'
        dataset_id = 'google'
        table_id = 'google-sites'
        full_table_id = f'{project_id}.{dataset_id}.{table_id}'

        #leemos los sitios que vamos a utilizar para luego cargar solo las reviews de estos sitios
        tabla = client.get_table(full_table_id)

        #guardamos la tabla en un dataframe
        df_sitios = client.list_rows(tabla).to_dataframe()

        df_filtrado = pd.merge(df, df_sitios[['gmap_id']], on='gmap_id', how='inner')

        print(f"Cantidad de filas de df_filtrado={df_filtrado.shape}")

        #CARGA EN BIG QUERY:
        client = bigquery.Client()

        # Especifica el nombre del conjunto de datos y la tabla

        # Configura el job de carga con autodetección de esquema
        job_config = bigquery.LoadJobConfig(
            autodetect=True,  # Permite a BigQuery autodetectar el esquema
            create_disposition="CREATE_IF_NEEDED",
            write_disposition="WRITE_APPEND" 
        )


        table_id = 'reviews'
        full_table_id = f'{project_id}.{dataset_id}.{table_id}'

        # Inicia el job de carga
        load_job = client.load_table_from_dataframe(df_filtrado, full_table_id, job_config=job_config, location="us-central1")

        # Espera a que el job se complete
        load_job.result()

    except Exception as e:
        logging.error(f"Error en la función: {e}", exc_info=True)